In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
import re
import math

In [4]:
df= pd.read_csv('main_task.xls')

In [5]:
df

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963
...,...,...,...,...,...,...,...,...,...,...
39995,id_499,Milan,"['Italian', 'Vegetarian Friendly', 'Vegan Opti...",500.0,4.5,$$ - $$$,79.0,"[['The real Italian experience!', 'Wonderful f...",/Restaurant_Review-g187849-d2104414-Reviews-Ro...,d2104414
39996,id_6340,Paris,"['French', 'American', 'Bar', 'European', 'Veg...",6341.0,3.5,$$ - $$$,542.0,"[['Parisian atmosphere', 'Bit pricey but inter...",/Restaurant_Review-g187147-d1800036-Reviews-La...,d1800036
39997,id_1649,Stockholm,"['Japanese', 'Sushi']",1652.0,4.5,NaN,4.0,"[['Good by swedish standards', 'A hidden jewel...",/Restaurant_Review-g189852-d947615-Reviews-Sus...,d947615
39998,id_640,Warsaw,"['Polish', 'European', 'Eastern European', 'Ce...",641.0,4.0,$$ - $$$,70.0,"[['Underground restaurant', 'Oldest Restaurant...",/Restaurant_Review-g274856-d1100838-Reviews-Ho...,d1100838


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Restaurant_id      40000 non-null  object 
 1   City               40000 non-null  object 
 2   Cuisine Style      30717 non-null  object 
 3   Ranking            40000 non-null  float64
 4   Rating             40000 non-null  float64
 5   Price Range        26114 non-null  object 
 6   Number of Reviews  37457 non-null  float64
 7   Reviews            40000 non-null  object 
 8   URL_TA             40000 non-null  object 
 9   ID_TA              40000 non-null  object 
dtypes: float64(3), object(7)
memory usage: 3.1+ MB


Очистка и разбивка данных

In [7]:
drop_list = ['Restaurant_id','City','Cuisine Style','Price Range','Reviews','URL_TA','ID_TA']
df_prep = df.drop(drop_list, axis = 1)

In [8]:
df_prep

,Ranking,Rating,Number of Reviews
0,5570.0,3.5,194.0
1,1537.0,4.0,10.0
2,353.0,4.5,688.0
3,3458.0,5.0,3.0
4,621.0,4.0,84.0
...,...,...,...
39995,500.0,4.5,79.0
39996,6341.0,3.5,542.0
39997,1652.0,4.5,4.0
39998,641.0,4.0,70.0


In [9]:
NoR_median_val = round(df_prep['Number of Reviews'].median())
df_prep.fillna({'Number of Reviews':NoR_median_val},inplace=True)

In [10]:
df_prep.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Ranking            40000 non-null  float64
 1   Rating             40000 non-null  float64
 2   Number of Reviews  40000 non-null  float64
dtypes: float64(3)
memory usage: 937.6 KB


In [11]:
df_prep['Number of Reviews'].value_counts()

33.0      2801
2.0       1916
3.0       1636
4.0       1370
5.0       1181
          ... 
2141.0       1
920.0        1
1404.0       1
979.0        1
1741.0       1
Name: Number of Reviews, Length: 1459, dtype: int64

In [12]:
# Х - данные с информацией о ресторанах, у - целевая переменная (рейтинги ресторанов)
X = df_prep.drop('Rating', axis = 1)
y = df_prep['Rating']

In [13]:
# Загружаем специальный инструмент для разбивки:
from sklearn.model_selection import train_test_split

In [14]:
# Наборы данных с меткой "train" будут использоваться для обучения модели, "test" - для тестирования.
# Для тестирования мы будем использовать 25% от исходного датасета.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [15]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [16]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [17]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.43419748683677434


In [18]:
df['Price Range'].value_counts()

$$ - $$$    18412
$            6279
$$$$         1423
Name: Price Range, dtype: int64

In [19]:
df['City'].value_counts()
df['City'].nunique()

31

In [20]:
df['Cuisine Style'].value_counts()

['Italian']                                                                                       1032
['French']                                                                                         805
['Spanish']                                                                                        695
['French', 'European']                                                                             405
['Cafe']                                                                                           403
                                                                                                  ... 
['Brazilian', 'South American', 'Vegetarian Friendly', 'Vegan Options', 'Gluten Free Options']       1
['American', 'European', 'Hungarian', 'Vegetarian Friendly']                                         1
['Italian', 'Bar', 'Mediterranean', 'European', 'Wine Bar', 'Brew Pub', 'Vegetarian Friendly']       1
['French', 'Cafe', 'European', 'Bar', 'Pub', 'Vegetarian Friendly']      

In [21]:
re.split('\W{2,4}', "['Sushi', 'Asian', 'Vietnamese', 'Seafood', 'Vegetarian Friendly']")

['', 'Sushi', 'Asian', 'Vietnamese', 'Seafood', 'Vegetarian Friendly', '']

In [22]:
cuisine_list = list()
cuisine_set = set()
for row in df['Cuisine Style']:
    if row == row:
        row_split = re.split('\W{4}', row[2:-2])
        cuisine_list.append(row_split)
        for name in row_split:
            cuisine_set.add(name)
    else:
        cuisine_list.append(['Vegetarian Friendly'])
cuisine_set

{'Afghani',
 'African',
 'Albanian',
 'American',
 'Arabic',
 'Argentinean',
 'Armenian',
 'Asian',
 'Australian',
 'Austrian',
 'Azerbaijani',
 'Balti',
 'Bangladeshi',
 'Bar',
 'Barbecue',
 'Belgian',
 'Brazilian',
 'Brew Pub',
 'British',
 'Burmese',
 'Cafe',
 'Cajun & Creole',
 'Cambodian',
 'Canadian',
 'Caribbean',
 'Caucasian',
 'Central American',
 'Central Asian',
 'Central European',
 'Chilean',
 'Chinese',
 'Colombian',
 'Contemporary',
 'Croatian',
 'Cuban',
 'Czech',
 'Danish',
 'Delicatessen',
 'Diner',
 'Dutch',
 'Eastern European',
 'Ecuadorean',
 'Egyptian',
 'Ethiopian',
 'European',
 'Fast Food',
 'Filipino',
 'French',
 'Fujian',
 'Fusion',
 'Gastropub',
 'Georgian',
 'German',
 'Gluten Free Options',
 'Greek',
 'Grill',
 'Halal',
 'Hawaiian',
 'Healthy',
 'Hungarian',
 'Indian',
 'Indonesian',
 'International',
 'Irish',
 'Israeli',
 'Italian',
 'Jamaican',
 'Japanese',
 'Korean',
 'Kosher',
 'Latin',
 'Latvian',
 'Lebanese',
 'Malaysian',
 'Mediterranean',
 'Mexic

In [23]:
len(cuisine_set)

125

In [24]:
df['cuisine_divided'] = cuisine_list 

In [25]:
df

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,cuisine_divided
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,"[European, French, International]"
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,[Vegetarian Friendly]
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,"[Japanese, Sushi, Asian, Grill, Vegetarian Fri..."
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776,[Vegetarian Friendly]
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963,"[German, Central European, Vegetarian Friendly]"
...,...,...,...,...,...,...,...,...,...,...,...
39995,id_499,Milan,"['Italian', 'Vegetarian Friendly', 'Vegan Opti...",500.0,4.5,$$ - $$$,79.0,"[['The real Italian experience!', 'Wonderful f...",/Restaurant_Review-g187849-d2104414-Reviews-Ro...,d2104414,"[Italian, Vegetarian Friendly, Vegan Options, ..."
39996,id_6340,Paris,"['French', 'American', 'Bar', 'European', 'Veg...",6341.0,3.5,$$ - $$$,542.0,"[['Parisian atmosphere', 'Bit pricey but inter...",/Restaurant_Review-g187147-d1800036-Reviews-La...,d1800036,"[French, American, Bar, European, Vegetarian F..."
39997,id_1649,Stockholm,"['Japanese', 'Sushi']",1652.0,4.5,NaN,4.0,"[['Good by swedish standards', 'A hidden jewel...",/Restaurant_Review-g189852-d947615-Reviews-Sus...,d947615,"[Japanese, Sushi]"
39998,id_640,Warsaw,"['Polish', 'European', 'Eastern European', 'Ce...",641.0,4.0,$$ - $$$,70.0,"[['Underground restaurant', 'Oldest Restaurant...",/Restaurant_Review-g274856-d1100838-Reviews-Ho...,d1100838,"[Polish, European, Eastern European, Central E..."


In [26]:
cuisine_pop_dict = dict.fromkeys(cuisine_set,0)
for row in df['cuisine_divided']:
    if row != None:
        for element in row:
            cuisine_pop_dict[element] =cuisine_pop_dict[element]+1
cuisine_pop_dict

{'Cafe': 2325,
 'Israeli': 74,
 'International': 1584,
 'Romanian': 10,
 'Tibetan': 24,
 'Spanish': 2798,
 'Caribbean': 93,
 'Nepali': 83,
 'Middle Eastern': 782,
 'Peruvian': 94,
 'Fujian': 2,
 'American': 1315,
 'Pizza': 2849,
 'New Zealand': 6,
 'European': 10060,
 'Chinese': 1145,
 'Latvian': 1,
 'Cajun & Creole': 20,
 'Bangladeshi': 76,
 'Brazilian': 127,
 'Steakhouse': 573,
 'Grill': 490,
 'Swedish': 171,
 'Georgian': 17,
 'Dutch': 294,
 'Eastern European': 496,
 'Halal': 598,
 'Fusion': 577,
 'Lebanese': 329,
 'Filipino': 12,
 'Australian': 24,
 'Xinjiang': 1,
 'Welsh': 2,
 'Ukrainian': 18,
 'Gastropub': 471,
 'Mexican': 445,
 'Afghani': 22,
 'Belgian': 270,
 'Balti': 83,
 'Venezuelan': 40,
 'Healthy': 620,
 'Armenian': 14,
 'Vietnamese': 513,
 'Brew Pub': 225,
 'Burmese': 1,
 'Central Asian': 12,
 'Bar': 3297,
 'Cuban': 23,
 'Ecuadorean': 4,
 'Seafood': 1505,
 'Polynesian': 2,
 'Russian': 58,
 'Kosher': 48,
 'Norwegian': 75,
 'Fast Food': 1705,
 'Uzbek': 5,
 'Slovenian': 68,
 '

In [27]:
import operator
max(cuisine_pop_dict.items(), key=operator.itemgetter(1))[0]

'Vegetarian Friendly'

In [28]:
cuisine_num = list()
for row in df['cuisine_divided']:
    if row != None:
        cuisine_num.append(len(row))
    else:
        cuisine_num.append(1)

In [29]:
df['cuisine_mean']=cuisine_num

df

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,cuisine_divided,cuisine_mean
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,$$ - $$$,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,"[European, French, International]",3
1,id_1535,Stockholm,NaN,1537.0,4.0,NaN,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,[Vegetarian Friendly],1
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,$$$$,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,"[Japanese, Sushi, Asian, Grill, Vegetarian Fri...",7
3,id_3456,Berlin,NaN,3458.0,5.0,NaN,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776,[Vegetarian Friendly],1
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,$$ - $$$,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963,"[German, Central European, Vegetarian Friendly]",3
...,...,...,...,...,...,...,...,...,...,...,...,...
39995,id_499,Milan,"['Italian', 'Vegetarian Friendly', 'Vegan Opti...",500.0,4.5,$$ - $$$,79.0,"[['The real Italian experience!', 'Wonderful f...",/Restaurant_Review-g187849-d2104414-Reviews-Ro...,d2104414,"[Italian, Vegetarian Friendly, Vegan Options, ...",4
39996,id_6340,Paris,"['French', 'American', 'Bar', 'European', 'Veg...",6341.0,3.5,$$ - $$$,542.0,"[['Parisian atmosphere', 'Bit pricey but inter...",/Restaurant_Review-g187147-d1800036-Reviews-La...,d1800036,"[French, American, Bar, European, Vegetarian F...",5
39997,id_1649,Stockholm,"['Japanese', 'Sushi']",1652.0,4.5,NaN,4.0,"[['Good by swedish standards', 'A hidden jewel...",/Restaurant_Review-g189852-d947615-Reviews-Sus...,d947615,"[Japanese, Sushi]",2
39998,id_640,Warsaw,"['Polish', 'European', 'Eastern European', 'Ce...",641.0,4.0,$$ - $$$,70.0,"[['Underground restaurant', 'Oldest Restaurant...",/Restaurant_Review-g274856-d1100838-Reviews-Ho...,d1100838,"[Polish, European, Eastern European, Central E...",5


In [30]:
df['cuisine_mean'].mean()

2.6224

In [31]:
df['Price Range'].fillna('$$ - $$$',inplace= True)

In [32]:
price_dict = dict()
price_dict['$']=1
price_dict['$$ - $$$']=2.5
price_dict['$$$$']=4

def price(row):
    return price_dict[row]
df['Price Range'] = df['Price Range'].apply(price)
display(df)

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,cuisine_divided,cuisine_mean
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,2.5,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,"[European, French, International]",3
1,id_1535,Stockholm,NaN,1537.0,4.0,2.5,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,[Vegetarian Friendly],1
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,4.0,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,"[Japanese, Sushi, Asian, Grill, Vegetarian Fri...",7
3,id_3456,Berlin,NaN,3458.0,5.0,2.5,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776,[Vegetarian Friendly],1
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,2.5,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963,"[German, Central European, Vegetarian Friendly]",3
...,...,...,...,...,...,...,...,...,...,...,...,...
39995,id_499,Milan,"['Italian', 'Vegetarian Friendly', 'Vegan Opti...",500.0,4.5,2.5,79.0,"[['The real Italian experience!', 'Wonderful f...",/Restaurant_Review-g187849-d2104414-Reviews-Ro...,d2104414,"[Italian, Vegetarian Friendly, Vegan Options, ...",4
39996,id_6340,Paris,"['French', 'American', 'Bar', 'European', 'Veg...",6341.0,3.5,2.5,542.0,"[['Parisian atmosphere', 'Bit pricey but inter...",/Restaurant_Review-g187147-d1800036-Reviews-La...,d1800036,"[French, American, Bar, European, Vegetarian F...",5
39997,id_1649,Stockholm,"['Japanese', 'Sushi']",1652.0,4.5,2.5,4.0,"[['Good by swedish standards', 'A hidden jewel...",/Restaurant_Review-g189852-d947615-Reviews-Sus...,d947615,"[Japanese, Sushi]",2
39998,id_640,Warsaw,"['Polish', 'European', 'Eastern European', 'Ce...",641.0,4.0,2.5,70.0,"[['Underground restaurant', 'Oldest Restaurant...",/Restaurant_Review-g274856-d1100838-Reviews-Ho...,d1100838,"[Polish, European, Eastern European, Central E...",5


In [33]:
revtime_list = list()
for row in df['Reviews']:
    if row == row:
        date = re.findall("\d{2}/\d{2}/\d{4}", row)
        revtime_list.append(date)
    else:
        revtime_list.append([[],[]])
    

In [34]:
revtime_list[3] !=[]

False

In [35]:
revtime_list_1 = list()
revtime_list_2 = list()
for element in revtime_list:
    if element != []:
        revtime_list_1.append(element[0])
        if len(element)<2:
            revtime_list_2.append(element[0])
        else:
            revtime_list_2.append(element[1])
    else:
        revtime_list_1.append(None)
        revtime_list_2.append(None)

In [36]:
df['rev_date_1']=revtime_list_1
df['rev_date_2']=revtime_list_2
revtime_list_1

['12/31/2017',
 '07/06/2017',
 '01/08/2018',
 None,
 '11/18/2017',
 '09/16/2017',
 '08/04/2017',
 '02/16/2014',
 None,
 '11/07/2017',
 None,
 '12/15/2017',
 '01/07/2018',
 '07/25/2017',
 None,
 '12/01/2017',
 '04/12/2017',
 '02/28/2017',
 '08/16/2017',
 '04/12/2016',
 '12/12/2017',
 None,
 '08/05/2017',
 '12/01/2017',
 '08/06/2017',
 None,
 '07/05/2016',
 '01/26/2017',
 '08/20/2011',
 None,
 '08/03/2017',
 '09/01/2017',
 '10/26/2015',
 None,
 '01/27/2017',
 None,
 '04/05/2015',
 None,
 '03/08/2015',
 '05/12/2017',
 None,
 '10/18/2017',
 '10/07/2017',
 '02/14/2017',
 None,
 '07/19/2017',
 None,
 '02/15/2018',
 '06/05/2017',
 '12/18/2017',
 '01/06/2018',
 '12/08/2017',
 None,
 '11/26/2017',
 '01/05/2018',
 '03/19/2016',
 '11/10/2014',
 '04/13/2012',
 '05/28/2017',
 None,
 '12/21/2017',
 None,
 '08/31/2016',
 '01/07/2018',
 '01/09/2018',
 '09/30/2017',
 '12/10/2017',
 '10/29/2016',
 None,
 '07/19/2017',
 '08/27/2017',
 '12/09/2014',
 '01/08/2018',
 None,
 '11/04/2017',
 '04/25/2015',
 '12

In [37]:
df.rev_date_1 = df.rev_date_1.apply(lambda x:pd.to_datetime(x))
df.rev_date_2 = df.rev_date_2.apply(lambda x:pd.to_datetime(x))

In [38]:
df.rev_date_1.max()
df.rev_date_2.max()

Timestamp('2018-02-26 00:00:00')

In [39]:
df['rev_diff']=abs(df['rev_date_1']-df['rev_date_2'])
df.rev_diff.max()

Timedelta('3207 days 00:00:00')

Проверим вероятность неверного ввода

In [40]:
print(df[df.rev_diff == df.rev_diff.max()]['Reviews'].iloc[0])

[['Incredible buffet!', 'Used to be great, but still good under new...'], ['10/02/2016', '12/22/2007']]


Промежуток между отзывами составляет около девяти-десяти лет, что не необычно для некоторых ресторанов, хотя сами отзывы обычно оставляются чаще. Разберемся, как часто это происходит в нашем датасете:

In [41]:
df.rev_diff.value_counts(bins=4,dropna=True)

(-4 days +19:01:55.199999999, 801 days 18:00:00]    32920
(801 days 18:00:00, 1603 days 12:00:00]               525
(1603 days 12:00:00, 2405 days 06:00:00]               67
(2405 days 06:00:00, 3207 days 00:00:00]               17
Name: rev_diff, dtype: int64

Как мы видим, в подавляющем большинстве ресторанов отзывы оказываются на сайте чаще, чем раз в два-три года. Тем не менее, для части ресторанов промежуток между отзывами является достаточно большим.

In [42]:
Rev_time_med = df['rev_diff'].median()
df['rev_diff'].fillna(Rev_time_med,inplace=True)
df

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,cuisine_divided,cuisine_mean,rev_date_1,rev_date_2,rev_diff
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,2.5,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,"[European, French, International]",3,2017-12-31,2017-11-20,41 days
1,id_1535,Stockholm,NaN,1537.0,4.0,2.5,10.0,"[['Unique cuisine', 'Delicious Nepalese food']...",/Restaurant_Review-g189852-d7992032-Reviews-Bu...,d7992032,[Vegetarian Friendly],1,2017-07-06,2016-06-19,382 days
2,id_352,London,"['Japanese', 'Sushi', 'Asian', 'Grill', 'Veget...",353.0,4.5,4.0,688.0,"[['Catch up with friends', 'Not exceptional'],...",/Restaurant_Review-g186338-d8632781-Reviews-RO...,d8632781,"[Japanese, Sushi, Asian, Grill, Vegetarian Fri...",7,2018-01-08,2018-01-06,2 days
3,id_3456,Berlin,NaN,3458.0,5.0,2.5,3.0,"[[], []]",/Restaurant_Review-g187323-d1358776-Reviews-Es...,d1358776,[Vegetarian Friendly],1,NaT,NaT,49 days
4,id_615,Munich,"['German', 'Central European', 'Vegetarian Fri...",621.0,4.0,2.5,84.0,"[['Best place to try a Bavarian food', 'Nice b...",/Restaurant_Review-g187309-d6864963-Reviews-Au...,d6864963,"[German, Central European, Vegetarian Friendly]",3,2017-11-18,2017-02-19,272 days
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,id_499,Milan,"['Italian', 'Vegetarian Friendly', 'Vegan Opti...",500.0,4.5,2.5,79.0,"[['The real Italian experience!', 'Wonderful f...",/Restaurant_Review-g187849-d2104414-Reviews-Ro...,d2104414,"[Italian, Vegetarian Friendly, Vegan Options, ...",4,2017-12-16,2017-11-12,34 days
39996,id_6340,Paris,"['French', 'American', 'Bar', 'European', 'Veg...",6341.0,3.5,2.5,542.0,"[['Parisian atmosphere', 'Bit pricey but inter...",/Restaurant_Review-g187147-d1800036-Reviews-La...,d1800036,"[French, American, Bar, European, Vegetarian F...",5,2017-12-21,2017-12-12,9 days
39997,id_1649,Stockholm,"['Japanese', 'Sushi']",1652.0,4.5,2.5,4.0,"[['Good by swedish standards', 'A hidden jewel...",/Restaurant_Review-g189852-d947615-Reviews-Sus...,d947615,"[Japanese, Sushi]",2,2016-11-03,2008-04-12,3127 days
39998,id_640,Warsaw,"['Polish', 'European', 'Eastern European', 'Ce...",641.0,4.0,2.5,70.0,"[['Underground restaurant', 'Oldest Restaurant...",/Restaurant_Review-g274856-d1100838-Reviews-Ho...,d1100838,"[Polish, European, Eastern European, Central E...",5,2017-07-11,2017-06-18,23 days


In [43]:
df.fillna({'Number of Reviews':NoR_median_val},inplace=True)

In [44]:
city_dict = dict()
city_array = df['City'].unique()
for element in city_array:
    city_dict[element]=0
for index, row in df.iterrows():
    city_dict[row['City']] =city_dict[row['City']]+row['Number of Reviews']
city_dict

{'Paris': 568836.0,
 'Stockholm': 61121.0,
 'London': 757656.0,
 'Berlin': 136518.0,
 'Munich': 69446.0,
 'Oporto': 66134.0,
 'Milan': 356036.0,
 'Bratislava': 15116.0,
 'Vienna': 92674.0,
 'Rome': 585930.0,
 'Barcelona': 343737.0,
 'Madrid': 298024.0,
 'Dublin': 100585.0,
 'Brussels': 90397.0,
 'Zurich': 40256.0,
 'Warsaw': 44644.0,
 'Budapest': 121382.0,
 'Copenhagen': 57686.0,
 'Amsterdam': 153559.0,
 'Lyon': 74501.0,
 'Hamburg': 43084.0,
 'Lisbon': 156510.0,
 'Prague': 164318.0,
 'Oslo': 35372.0,
 'Helsinki': 22044.0,
 'Edinburgh': 134954.0,
 'Geneva': 28948.0,
 'Ljubljana': 13141.0,
 'Athens': 59326.0,
 'Luxembourg': 16613.0,
 'Krakow': 50959.0}

In [45]:
popularity_list = list()
for index, row in df.iterrows():
    popularity_list.append((row['Number of Reviews']/city_dict[row['City']]))
npa = np.asarray(popularity_list, dtype=np.float32)
#scaler = MinMaxScaler()
#npa = scaler.fit_transform(npa.reshape(-1,1))
#npa

In [46]:
df['Popularity']=npa

In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype          
---  ------             --------------  -----          
 0   Restaurant_id      40000 non-null  object         
 1   City               40000 non-null  object         
 2   Cuisine Style      30717 non-null  object         
 3   Ranking            40000 non-null  float64        
 4   Rating             40000 non-null  float64        
 5   Price Range        40000 non-null  float64        
 6   Number of Reviews  40000 non-null  float64        
 7   Reviews            40000 non-null  object         
 8   URL_TA             40000 non-null  object         
 9   ID_TA              40000 non-null  object         
 10  cuisine_divided    40000 non-null  object         
 11  cuisine_mean       40000 non-null  int64          
 12  rev_date_1         33529 non-null  datetime64[ns] 
 13  rev_date_2         33529 non-null  datetime64[

In [48]:
df['rev_to_today']=(pd.to_datetime('26.02.2018')-df['rev_date_1'])

In [49]:
df['rev_diff'] = df['rev_diff'].apply(lambda x: x.days)
df['rev_to_today']=df['rev_to_today'].apply(lambda x:x.days)
Rev_today = df['rev_to_today'].median()
df['rev_to_today'].fillna(Rev_today,inplace=True)

In [50]:
df_parse_data = pd.read_csv('parse_data.csv')
df_parse_data.columns = ['City','population','area','density']
display(df_parse_data)

,City,population,area,density
0,Paris,10785092,105,3800
1,Stockholm,1605030,188,4200
2,London,9787426,1572,5666
3,Berlin,4473100,891,5020
4,Munich,2606021,310,4800
5,Oporto,287591,41,6900
6,Milan,1399860,181,7700
7,Bratislava,563682,367,661
8,Vienna,1911191,414,4326
9,Rome,4342212,1285,2236


In [51]:
df = df.merge(df_parse_data, on='City')
df

,Restaurant_id,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA,cuisine_divided,cuisine_mean,rev_date_1,rev_date_2,rev_diff,Popularity,rev_to_today,population,area,density
0,id_5569,Paris,"['European', 'French', 'International']",5570.0,3.5,2.5,194.0,"[['Good food at your doorstep', 'A good hotel ...",/Restaurant_Review-g187147-d1912643-Reviews-R_...,d1912643,"[European, French, International]",3,2017-12-31,2017-11-20,41,0.000341,57.0,10785092,105,3800
1,id_10057,Paris,NaN,10059.0,4.5,2.5,4.0,"[['Fab Food'], ['08/20/2011']]",/Restaurant_Review-g187147-d2040769-Reviews-Le...,d2040769,[Vegetarian Friendly],1,2011-08-20,2011-08-20,0,0.000007,2382.0,10785092,105,3800
2,id_11291,Paris,['French'],11293.0,4.5,2.5,10.0,[['A relaxing dinner in a very personal atmos....,/Restaurant_Review-g187147-d3543081-Reviews-La...,d3543081,[French],1,2015-03-08,2015-03-08,0,0.000018,1086.0,10785092,105,3800
3,id_5459,Paris,"['Asian', 'Thai', 'Vegetarian Friendly']",5460.0,3.5,2.5,132.0,[['Very good Thai Food but small Portions and....,/Restaurant_Review-g187147-d953033-Reviews-Cha...,d953033,"[Asian, Thai, Vegetarian Friendly]",3,2017-10-18,2017-07-29,81,0.000232,131.0,10785092,105,3800
4,id_9924,Paris,['Pub'],9926.0,3.5,1.0,39.0,"[['English pub', 'Shameful'], ['02/14/2017', '...",/Restaurant_Review-g187147-d5799345-Reviews-Th...,d5799345,[Pub],1,2017-02-14,2017-12-10,299,0.000069,377.0,10785092,105,3800
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,id_651,Krakow,"['Italian', 'Pizza']",652.0,4.0,2.5,28.0,"[['Warm and welcoming', 'Cozy, good food'], ['...",/Restaurant_Review-g274772-d3626236-Reviews-Tr...,d3626236,"[Italian, Pizza]",2,2017-10-10,2015-10-15,726,0.000549,139.0,779115,326,2359
39996,id_355,Krakow,"['Italian', 'Mediterranean', 'Vegetarian Frien...",356.0,4.0,2.5,201.0,"[['Nice meal', 'Excellent food'], ['11/12/2017...",/Restaurant_Review-g274772-d1929643-Reviews-La...,d1929643,"[Italian, Mediterranean, Vegetarian Friendly]",3,2017-11-12,2017-06-28,137,0.003944,106.0,779115,326,2359
39997,id_283,Krakow,"['Italian', 'Pizza', 'European', 'Vegetarian F...",284.0,4.0,2.5,340.0,"[['Good food and friendly service', 'Nice, fri...",/Restaurant_Review-g274772-d1098691-Reviews-Fa...,d1098691,"[Italian, Pizza, European, Vegetarian Friendly...",5,2017-12-20,2017-10-23,58,0.006672,68.0,779115,326,2359
39998,id_800,Krakow,NaN,801.0,4.5,2.5,11.0,"[['Great snacks', ""You won't be disappointed""]...",/Restaurant_Review-g274772-d3899509-Reviews-Pi...,d3899509,[Vegetarian Friendly],1,2016-10-05,2016-06-08,119,0.000216,509.0,779115,326,2359


In [52]:
#pol_col = ['Ranking','Price Range','Number of Reviews', 'cuisine_mean','rev_diff','rev_to_today','Popularity']
#pf = PolynomialFeatures(2, interaction_only=True)
#poly_features = pf.fit_transform(df[pol_col])
#df_poly = pd.DataFrame(data=poly_features)
#df = df.merge(df_poly, left_index=True, right_index=True)

In [53]:
df = pd.get_dummies(df, columns=[ 'City'], dummy_na=True)
df_cuisine = pd.get_dummies(df.cuisine_divided.apply(pd.Series).stack()).sum(level=0)

In [54]:
df = df.merge(df_cuisine, left_index=True, right_index=True)

In [55]:

df_prep_2 = df.drop(['Restaurant_id','Cuisine Style','Reviews','URL_TA','ID_TA','cuisine_divided'],axis = 1)
df_prep_2 = df_prep_2.drop(['rev_date_1','rev_date_2'],axis = 1)
X = df_prep_2.drop('Rating', axis = 1)
y = df_prep_2['Rating']
df_prep_2

,Ranking,Rating,Price Range,Number of Reviews,cuisine_mean,rev_diff,Popularity,rev_to_today,population,area,...,Ukrainian,Uzbek,Vegan Options,Vegetarian Friendly,Venezuelan,Vietnamese,Welsh,Wine Bar,Xinjiang,Yunnan
0,5570.0,3.5,2.5,194.0,3,41,0.000341,57.0,10785092,105,...,0,0,0,0,0,0,0,0,0,0
1,10059.0,4.5,2.5,4.0,1,0,0.000007,2382.0,10785092,105,...,0,0,0,1,0,0,0,0,0,0
2,11293.0,4.5,2.5,10.0,1,0,0.000018,1086.0,10785092,105,...,0,0,0,0,0,0,0,0,0,0
3,5460.0,3.5,2.5,132.0,3,81,0.000232,131.0,10785092,105,...,0,0,0,1,0,0,0,0,0,0
4,9926.0,3.5,1.0,39.0,1,299,0.000069,377.0,10785092,105,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,652.0,4.0,2.5,28.0,2,726,0.000549,139.0,779115,326,...,0,0,0,0,0,0,0,0,0,0
39996,356.0,4.0,2.5,201.0,3,137,0.003944,106.0,779115,326,...,0,0,0,1,0,0,0,0,0,0
39997,284.0,4.0,2.5,340.0,5,58,0.006672,68.0,779115,326,...,0,0,1,1,0,0,0,0,0,0
39998,801.0,4.5,2.5,11.0,1,119,0.000216,509.0,779115,326,...,0,0,0,1,0,0,0,0,0,0


In [56]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [57]:
# Создаём модель
regr = RandomForestRegressor(n_estimators=100)

# Обучаем модель на тестовом наборе данных
regr.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = regr.predict(X_test)

In [58]:
y_pred

array([4.84 , 3.295, 3.525, ..., 3.8  , 4.475, 3.915])

In [59]:
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.210238


Так как "Рейтинг" может принимать только определенный набор значений:

In [60]:
df.Rating.value_counts()

4.0    13692
4.5    10929
3.5     6824
5.0     3879
3.0     2955
2.5      915
2.0      466
1.0      201
1.5      139
Name: Rating, dtype: int64

Дообработаем результат предсказания модели:

In [61]:
y_pred = np.round_(y_pred*2,decimals=0)/2

In [62]:
print(y_pred)

[5.  3.5 3.5 ... 4.  4.5 4. ]


Что позволит дополнительно снизить абсолютное среднее отклонение:

In [63]:
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

MAE: 0.1785
